In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

device = 'cuda:0'
model_name = 'google/flan-t5-xxl'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = 'Context: '
inputs = tokenizer(prompt, return_tensors="pt")
inputs.input_ids.v

In [ ]:
# print(tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True))
print(tokenizer.decode([1193, 6327, 10, 3], skip_special_tokens=False))


In [ ]:
from data_preprocessing.parallel_processing.TVQA_eval import TVQA_eval

tvqa_eval = TVQA_eval()

In [ ]:
# embeddings = tvqa_eval.get_clip_embed_from_vid_name("s08e06_seg02_clip_17")
# print(len(embeddings))
# print(embeddings[0].shape)
# working ^^ 

In [ ]:
for i, question in enumerate(tvqa_eval.train_qa_json):
  # if i != 3:
  #   continue
  print(type(question))
  print(question)
  vecs, ans = tvqa_eval.create_context_vectors(question)
  ans_list = tvqa_eval.get_answers_from_question(question)

  print(question)
  print(vecs)
  print(vecs[0].shape)
  print(ans)
  print(ans_list)

  if i > 5:
    break


In [ ]:
list(zip(vecs, ans_list))

In [ ]:
tvqa_eval.get_subtitle_from_clip('s08e06_seg02_clip_17', 0, 1)

In [ ]:
tvqa_eval.get_subtitle_from_clip('met_s06e09_seg02_clip_15', 0, 1)

In [ ]:
print(tvqa_eval.subtitles[0])

In [ ]:
print(tvqa_eval.train_qa_json[0])

In [12]:
import torch
ones = torch.ones((1024))
ones

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [ ]:
context_vector = torch.ones((1024))
context_vector = torch.ones((1024))
attn_mask_arr[ != -100] = 1

In [10]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
huggingface_model_name = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name, return_special_tokens_mask=True)
label_tokenized = tokenizer("yes", padding=True, truncation=True).input_ids
label_tokenized

[4273, 1]

# DEEPKAE

In [1]:
import deeplake as dl
import numpy as np
ds = dl.load("/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_tvqa_whole", read_only=True)
ds.summary()

print(ds.size_approx() / 1e9) # bytes to GB


/home/k/utils/miniconda3/envs/vpt/lib/python3.8/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.2.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_tvqa_whole loaded successfully.

Dataset(path='/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_tvqa_whole', read_only=True, tensors=['context_vector', 'label'])

     tensor       htype          shape           dtype  compression
    -------      -------        -------         -------  ------- 
 context_vector  generic  (117730, 1024, 1024)  float32   None   
     label        text        (117730, 1)         str     None   
941.866


In [18]:
for i, val in enumerate(ds):
  print(val.context_vector.numpy().shape)
  print(val.label.text())
  if i > 20: break

(1024, 1024)
yes
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
yes
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
yes
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
yes
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no
(1024, 1024)
no


In [3]:
import deeplake as dl
import numpy as np

BATCH_NAME = 'tvqa_whole'
RESULTS_DATASET_PATH = f'/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/mar_28_TVQA_encode_{BATCH_NAME}'
output_ds = dl.empty(RESULTS_DATASET_PATH, overwrite=True)

with output_ds:
  output_ds.create_tensor("context_vector", htype="generic", dtype=np.float32, sample_compression=None)
  output_ds.create_tensor("label", htype="text", dtype=str, sample_compression=None)
  output_ds.flush()

In [ ]:
import os
import pprint
from enum import Enum

import accelerate
import lovely_tensors as lt
import numpy as np
import torch
from transformers import T5EncoderModel, T5Tokenizer, logging

lt.monkey_patch()

# suppress: Some weights of the model checkpoint at google/flan-t5-large were not used when initializing model.
# This is expected because we're initializing the encoder-only. So the decoder weights are not used.
logging.set_verbosity_error()

# TODO: Set max_restarts and max_task_retries to enable retry when the task crashes due to OOM.
os.environ["RAY_memory_monitor_refresh_ms"] = "0"  # prevents ray from killing the process when it runs out of memory
os.environ['TRANSFORMERS_CACHE'] = '/mnt/teton/utils/cache/huggingface'
os.environ['HF_DATASETS_CACHE'] = '/mnt/teton/utils/cache/datasets'

# chosen_datatype = torch.float16
chosen_datatype = torch.float32
model_name = "google/flan-t5-large"


class FlanT5Encoder:

  def __init__(self, device: str = "cuda:1"):
    self.device = device if torch.cuda.is_available() else "cpu"
    print("In FlanT5Encoder", self.device)
    self.tokenizer = T5Tokenizer.from_pretrained(model_name)
    self.model = T5EncoderModel.from_pretrained(model_name, torch_dtype=chosen_datatype).to(self.device)
    # self.model = T5EncoderModel.from_pretrained(
    #     "google/flan-t5-large",
    #     # set device_map to only cuda:0
    #     # device_map="sequential",
    #     torch_dtype=torch.float16).to(device)  # I would really like to use this, but I think it's not supported by numpy or something.
    # self.model = T5EncoderModel.from_pretrained("google/flan-t5-large",
    #                                             device_map="auto",
    #                                             torch_dtype=torch.float16,
    #                                             max_memory={
    #                                                 0: "24GiB",
    #                                                 1: "0GiB",
    #                                             })

  def encode(self, input_dict):  #batch):
    """
    OLD param batch: list of {'db_index': int, 'caption': str}
    param input_dict: dict of {'db_index': int, 'caption': str}. batch_size is 1
    return: list of np.arrays, each of different shape [NUM_TOKENS, 1024]

    Important to pad to "max_length" so we can stack the inputs.
    keeping truncation=False for now because we really don't expect to go over length (with 15-word sequences), and I want to see errors if we do.
    """
    last_hidden_states_batch = []
    # for input_dict in batch:
    with torch.inference_mode():
      # print("ABOUT TO ENCODE CAPTION: ", input_dict["caption"])
      tokens = self.tokenizer(input_dict["caption"], return_tensors="pt", padding=False, truncation=False).to(self.device)
      # print("TOKENS: ", tokens)
      lhs = self.model(**tokens).last_hidden_state

      # CAST FROM 32 to 16 bit via .half() !!
      lhs = lhs.half().detach().cpu().numpy().reshape(-1, 1024)
      # print("LAST HIDDEN STATE: ", lhs)
      last_hidden_states_batch.append({'last_hidden_states': lhs, 'db_index': input_dict['db_index']})
    # return: list of np.arrays, each of different shape [NUM_TOKENS, 1024]
    return last_hidden_states_batch

  def encode_tvqa(self, sentence, truncate_shape=804):

    def pad_or_truncate_tensor(tensor):
      target_shape = [truncate_shape, 1024]
      tensor_shape = tensor.shape

      # If tensor shape is larger than the target shape, truncate the tensor
      if tensor_shape[0] > target_shape[0]:
        truncated_tensor = tensor[:target_shape[0], :]
        return truncated_tensor

      # If tensor shape is smaller than the target shape, pad the tensor
      elif tensor_shape[0] < target_shape[0]:
        padding_shape = (target_shape[0] - tensor_shape[0], target_shape[1])
        padded_tensor = torch.nn.functional.pad(tensor, (0, 0, 0, padding_shape[0]), value=-100)
        return padded_tensor

      # If tensor shape is already the target shape, return the tensor
      else:
        return tensor

    # Tokenize the sentence and convert it to a PyTorch tensor
    tokens = self.tokenizer(sentence, return_tensors="pt", padding=False, truncation=False).to(self.device)

    # Generate the last hidden layer of the CLIP encoder
    lhs = self.model(**tokens).last_hidden_state

    # Truncate or pad last hidden states
    truncated_states = pad_or_truncate_tensor(lhs.squeeze(0))
    truncated_states = truncated_states.cpu()
    new_tensor = truncated_states
    if model_name == "google/flan-t5-small":
      new_tensor = torch.full((truncated_states.shape[0], 1024), -100)
      # Copy the original tensor's values to the first 512 columns of the new tensor
      new_tensor[:, :512] = truncated_states
    return new_tensor


In [ ]:
d = FlanT5Encoder()

In [ ]:
res = d.encode_tvqa("hello world")
res